In [19]:
# force the notebook to reload external python modules; useful for development
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# imports
import sys
import os
import pandas as pd
import numpy as np
import torch
import torchaudio
import json

sys.path.append("..\\")
from src.features import mfcc
from src.features import landmarks
from src.features import adjacency

In [16]:
# paths
raw_data_path = "..\\data\\raw\\"
processed_data_path = "..\\data\\processed\\"

In [ ]:
# load data
class MyDataset():
    def __init__(self, location=raw_data_path):
        self.location = location
        self.data_path = self.location + "dataset\\"
        self.general_labels = pd.read_csv(self.location + "labels.csv", index_col=0)
        self.labels = list()
        self.data = self.load_data()
    
    def load_data(self):
        _data = []
        dirs = os.listdir(self.data_path)
        for dir in dirs:
            files = os.listdir(self.data_path + dir)
            for i in range(1, 11):
                # check if [1-10].wav file exists
                if str(i) + ".wav" in files:
                    # append label to labels list
                    self.labels.append(self.general_labels.loc[dir][0])
                    # generate MFCCs for .wav file
                    frame = mfcc.process_audio_file(self.data_path + dir + "\\" + str(i) + ".wav")
                    # open .7z file and extract all .ljson files to create a list of landmarks
                    sequence = landmarks.get_mouth_sequences(self.data_path + dir + "\\" + str(i) + ".7z")
                    # create adjacency matrix
                    # IMPLIMENT THIS
                    # finally save
                    _data.append([frame, sequence])

        return _data

In [17]:
temp_loc = raw_data_path + "dataset\\A\\"

In [9]:
frame = mfcc.process_audio_file(temp_loc + "1.wav")

c:\Users\hgnis\anaconda3\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [11]:
frame.shape

torch.Size([1, 250, 400])

In [29]:
lm = landmarks.get_mouth_sequences(temp_loc + "1.7z")

In [30]:
lm.shape

(3, 250, 20, 1)